# Stage 1: Image Processing

Denoising and deblurring with some classical methods.

## Demo for classical method
- median filter;
- bilateral filter;
- wiener filter;
- wavelet transform (dummy approach);
- LMG kernal estimation

In [1]:
# append the path of the project to the system path
import os
import sys
path = os.getcwd()
sys.path.append(path)

### Grid Search for different noise vs method

In [ ]:
from src.stage1_test import test_dummy_stage1
import pandas as pd

noise = ['gaussian_blur', 'sp_noise', 'gaussian_blur', 'speckle_noise','motion_blur']
method = ['median', 'bilateral', 'wiener', 'wavelet', 'lmg']
stats = pd.DataFrame(columns=['noise_type', 'method', 'psnr', 'ssim'])

for n in noise:
    for m in method:
        stat = test_dummy_stage1(n, m)
        stats = stats.append(pd.Series(stat, index=stats.columns), ignore_index=True)

In [ ]:
print(stats)

## Test on Motion blur kernal

In [ ]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.signal import convolve2d

from src.utils import load_images
from PIL import Image, ImageOps

In [ ]:
kernel_size = np.arange(3, 21, 1)  # adjust as needed
angle = np.arange(0, 360, 0.5)  # adjust as needed

best_psnr = 0
best_ssim = 0

In [ ]:
images = load_images(path+'/stage1_data/input_imgs')
image = np.array(ImageOps.grayscale(Image.open(next(images))))/255.0

ref_images  = load_images(path+'/stage1_data/motion_blur')
ref_image = np.array(ImageOps.grayscale(Image.open(next(ref_images))))/255.0

In [ ]:
# Generate the motion blur kernel
for ks in kernel_size:
    for a in angle:
        motion_blur_kernel = np.zeros((ks, ks))
        motion_blur_kernel[int((ks-1)/2), :] = np.ones(ks)
        rotation_matrix = cv2.getRotationMatrix2D((int((ks-1)/2), int((ks-1)/2)), a, 1)
        motion_blur_kernel = cv2.warpAffine(motion_blur_kernel, rotation_matrix, (ks, ks))
        # Normalize the kernel
        motion_blur_kernel /= motion_blur_kernel.sum()
        # Apply the kernel to the input image
        image_blurred = convolve2d(image, motion_blur_kernel, mode='same', boundary='symm')
        # compute psnr and ssim
        psnr_value = psnr(image, image_blurred)
        ssim_value = ssim(image, image_blurred, multichannel=True, data_range=image_blurred.max() - image_blurred.min())
        if psnr_value > best_psnr:
            best_psnr = psnr_value
            best_ssim = ssim_value
            best_kernel = motion_blur_kernel
